In [ ]:
from dbcls_llm.client import DBCLSLLMClient
from dbcls_llm.config import MODELS

In [ ]:
print(MODELS)

In [ ]:
def make_client(model):
    return DBCLSLLMClient(username="*", password="*", model_info=model)

In [ ]:
DBCLSLLMClient.VERSION

In [ ]:
clients = {}
models = list(MODELS.keys())
models.remove('gemini')
for key in models:
    if 'gpt' in key:
        clients[key] = make_client(MODELS[key][0])
    else:
        for model in MODELS[key]:
          clients[key+model['model']] = make_client(model)

In [ ]:
clients

In [ ]:
def query(client, prompt):
    try:
        res = client.query(prompt).text
    except Exception as e:
        res = ""
    return res

In [ ]:
from concurrent.futures import ThreadPoolExecutor

def query_client(cp):
    return query(cp[0], cp[1])

prompt1 = 'hi'
results = []
#Using ThreadPoolExecutor to run encoding in parallel
#with ThreadPoolExecutor() as executor:
#    results = dict(zip(
#        clients.keys(), list(executor.map(query_client, zip(clients.values(), [prompt1 for x in range(len(clients))])))))
# for client in clients.values():
#     results.append(query(client, prompt1))

In [ ]:
results

In [ ]:
def query_all(prompt):
    with ThreadPoolExecutor() as executor:
        results = dict(zip(
            clients.keys(), list(executor.map(query_client, zip(clients.values(), [prompt for x in range(len(clients))])))))
    return results

In [ ]:
query_all('hi')

In [ ]:
from datasets import load_dataset

ds = load_dataset("allenai/WildChat-1M")


In [ ]:
%timeit initial_prompts = [d[0]['content'] for d in ds['train'][:]['conversation']]

In [ ]:
with open('questions.pkl', 'rb') as f:  # Open file in read-binary mode
    questions = pickle.load(f) 

In [ ]:
len(questions)

In [ ]:
KEEP_MSG_KEYS = ["bioinformatics","biology","microbiology","notype","genus","phylum","taxonomy","Prokaryote ","Bacteria","fungi","fungal","virus","Eukaryote"," gene "]

In [ ]:
filtered_questions = []
for question in questions:
    found = False
    for keyword in KEEP_MSG_KEYS:
        if keyword in question:
            found = True
    filtered_questions.append((found, question))

In [ ]:
len(filtered_questions)

In [ ]:
with open('data_array.pkl', 'rb') as f:  # Open file in read-binary mode
    loaded_data = pickle.load(f) 

In [ ]:
len(loaded_data)

In [ ]:
res = query_all('say hi back')

In [ ]:
res

In [ ]:

import logging
logging.basicConfig()
logging.getLogger().setLevel(logging.DEBUG)

In [ ]:

comparing_output = []
i = 1445+18
print(len(filtered_questions))
for question in filtered_questions[i:]:
    res = {}
    print(i)
    if question[0]:
        try:
            if question[0]:
                res = query_all(question[1])
            comparing_output.append((question[0], question[1], res))
        except Exception as e:
            print(f"An error occurred with question index {i}: {e}")
            # Optionally, append a default value or None for error cases
            comparing_output.append((question[0], question[1], {}))
    else:
        comparing_output.append((question[0], question[1], res))
    i += 1

In [ ]:
len(comparing_output)

In [ ]:
import pickle
with open('benchmark.pkl', 'wb') as f:  # Open file in write-binary mode
    pickle.dump(comparing_output, f) 

In [ ]:
comparing_output

In [ ]:
len(comparing_output)

In [ ]:
filtered_output = [x for x in comparing_output if x[2] != {}]

In [ ]:
len(filtered_output)

In [ ]:
filtered_output[0][2]

In [ ]:
%timeit allp = [(d[0]['content'], d[1]['content']) for d in ds['train'][:]['conversation']]

In [ ]:
tagged_conv = []
for conv in allp:
    found = False
    for keyword in KEEP_MSG_KEYS:
        if keyword in conv[0]:
            found = True
    tagged_conv.append((found, conv[0], conv[1]))

In [ ]:
bds = []


In [ ]:
i = 41941
print(len(ds['train']))
for row in ds['train']:
    print(i)
    prompt = row['conversation'][0]['content']
    res = {}
    found = False
    for keyword in KEEP_MSG_KEYS:
        if keyword in prompt:
            found = True
    if found:
        try:
            res = query_all(prompt)
        except Exception as e:
            print(f"An error occurred with question index {i}: {e}")
    row['compare'] = res
    bds.append(row)
    i += 1

In [ ]:
import pickle

In [ ]:
import pickle
with open('benchmark.pkl', 'wb') as f:  # Open file in write-binary mode
    pickle.dump(bds, f) 

In [ ]:
bds[168]['compare']['llamallama2-70b']

In [ ]:
all_comparisons = bds[168]['compare']
just_one_response = bds[168]['compare']['llamallama2-70b']
original_prompt = bds[168]['conversation'][0]['content']
original_response = bds[168]['conversation'][1]['content']

In [ ]:
len(bds)

In [ ]:
import pickle
with open('benchmark_lg.pkl', 'wb') as f:  # Open file in write-binary mode
    pickle.dump(bds, f) 

In [2]:
import Levenshtein

In [3]:
#!pip install sentence-transformers
from sentence_transformers import SentenceTransformer

# noinspection PyPackageRequirements
model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")

/Users/davidcs/git/dbcls_llm/env/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
i = 96009
for row in bdx[i:]:
    print(i)
    #row['prompt'] = row['conversation'][0]['content']
    row['prompt-embed'] = model.encode(row['conversation'][0]['content'])
    #row['response'] = row['conversation'][1]['content']
    row['response-embed'] = model.encode(row['conversation'][1]['content'])
    # print(row)
    row['embeddings'] = {}
    i+= 1
    if row['compare'] != {}:
        for key in row['compare'].keys():
            if row['compare'].get(key, None):
                row['embeddings'][key] = model.encode(row['compare'][key])
        # calculate distance from original response
        row['response-ldistances'] = {}
        row['response-sdistances'] = {}
        row['response-similarity'] = {}
        for key in row['embeddings'].keys():
            row['response-ldistances'][key] = Levenshtein.distance(row['conversation'][1]['content'], row['compare'][key])
            if min(len(row['conversation'][1]['content']), len(row['compare'][key])) > 0:
                max_len = max(len(row['conversation'][1]['content']), len(row['compare'][key]))
                similarity = 1 - (row['response-ldistances'][key] / max_len)
                row['response-similarity'][key] = similarity
            else:
                row['response-similarity'][key] = 0
            # embedded distance
            row['response-sdistances'][key] = sum(row['embeddings'][key] - row['response-embed'])

In [ ]:
!pip install python-Levenshtein

In [ ]:
bdx[168]['response-similarity']

In [1]:
import pickle
with open('benchmark_lg.pkl', 'rb') as f:  # Open file in write-binary mode
    bdx = pickle.load(f) 

In [ ]:
import pickle
with open('benchmark_lg_embed-distance-2.pkl', 'wb') as f:  # Open file in write-binary mode
    pickle.dump(bdx, f) 

In [ ]:
import pickle
with open('benchmark_lg_embed-distance.pkl', 'rb') as f:  # Open file in write-binary mode
    bdv = pickle.load(f) 

In [17]:
i = 0
for row in bdx:
    if row.get('embeddings', {}) != {}:
        i+=1
print(i)

571


In [ ]:
import pickle
with open('benchmark_326800_embed-distance.pkl', 'wb') as f:  # Open file in write-binary mode
    pickle.dump(bdx, f) 

In [13]:
bdx[168]['response-sdistances']

{'gpt-35-turbo': np.float32(-0.16393547),
 'gpt-4o': np.float32(-0.22431158),
 'gpt-4': np.float32(-0.084557414),
 'claude3opus': np.float32(-0.32143763),
 'claude3sonnet5': np.float32(-0.26128024),
 'claude3sonnet': np.float32(0.42277068),
 'claude3haiku': np.float32(-0.018462598),
 'commandrplus': np.float32(-0.054872647),
 'commandrbasic': np.float32(-0.13524169),
 'llamallama3-70b': np.float32(1.6736692),
 'llamallama3-8b': np.float32(-0.12971321),
 'llamallama2-70b': np.float32(0.055728346),
 'llamallama2-13b': np.float32(0.39503032),
 'mistralmistral-7b': np.float32(-0.2246772),
 'mistralmixtral-8x7b': np.float32(-0.07445856),
 'mistralmistral-large': np.float32(0.14975908)}

In [15]:
print('hi')

hi


In [ ]:
# last finished amount
# 326800